In [18]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('/JUET/Projects/reunion-data-assignment/Problem1/ecommerce.db')

In [19]:
# 1. Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months. The average order 
#    amount should be calculated for each customer, and the result should be sorted in descending order.

cursor = conn.cursor()
cursor.execute('''
    SELECT Customers.customer_id, Customers.name, AVG(Orders.total_amount) AS average_order_amount
    FROM Customers
    JOIN Orders ON Customers.customer_id = Orders.customer_id
    WHERE Orders.order_date >= DATE('now', '-6 months')
    GROUP BY Customers.customer_id, Customers.name
    ORDER BY average_order_amount DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[(4, 'Marcus Molina', 942.97), (10, 'Melissa Gaines', 711.53), (9, 'Chloe Rivas', 607.58), (5, 'Mandy Clark', 572.9300000000001), (7, 'Matthew Mccall', 568.2066666666666)]


In [20]:
# 2. Retrieve the list of customer whose order value is lower this year as compared to previous year

cursor = conn.cursor()
cursor.execute('''
    SELECT Customers.customer_id, Customers.name
    FROM Customers
    JOIN Orders ON Customers.customer_id = Orders.customer_id
    WHERE strftime('%Y', Orders.order_date) = strftime('%Y', 'now')
    AND Orders.total_amount < (
        SELECT SUM(total_amount)
        FROM Orders
        WHERE strftime('%Y', order_date) = strftime('%Y', 'now', '-1 year')
        AND Customers.customer_id = Orders.customer_id
    );
    ''')

query = cursor.fetchall()
print(query)

[(6, 'Matthew Hernandez'), (2, 'Miranda Walls'), (3, 'Brent Bailey'), (3, 'Brent Bailey'), (7, 'Matthew Mccall'), (5, 'Mandy Clark'), (2, 'Miranda Walls'), (6, 'Matthew Hernandez'), (7, 'Matthew Mccall'), (5, 'Mandy Clark'), (8, 'Katelyn Stewart'), (3, 'Brent Bailey'), (3, 'Brent Bailey'), (9, 'Chloe Rivas'), (2, 'Miranda Walls'), (4, 'Marcus Molina'), (2, 'Miranda Walls'), (9, 'Chloe Rivas'), (2, 'Miranda Walls'), (8, 'Katelyn Stewart'), (6, 'Matthew Hernandez'), (5, 'Mandy Clark'), (3, 'Brent Bailey'), (2, 'Miranda Walls'), (7, 'Matthew Mccall'), (3, 'Brent Bailey'), (6, 'Matthew Hernandez'), (5, 'Mandy Clark'), (10, 'Melissa Gaines'), (9, 'Chloe Rivas'), (1, 'Ian Hall'), (2, 'Miranda Walls')]


In [21]:
# 3. Create a table showing cumulative purchase by a particular customer

cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE CumulativePurchase AS
    SELECT Orders.order_id, Orders.customer_id, Orders.order_date, SUM(Orders.total_amount) OVER (PARTITION BY Orders.customer_id ORDER BY Orders.order_date) AS cumulative_purchase
    FROM Orders;
    ''')

cursor.execute("SELECT * FROM CumulativePurchase;")
query = cursor.fetchall()
print(query)

[(53, 1, '2021-12-19', 198.42), (52, 1, '2022-02-21', 892.93), (24, 1, '2022-05-10', 1070.21), (13, 1, '2022-07-01', 1323.67), (47, 1, '2022-08-24', 1665.3600000000001), (21, 1, '2022-09-22', 2633.9), (92, 1, '2023-03-08', 2695.58), (67, 2, '2022-01-04', 471.35), (97, 2, '2022-01-06', 1003.7900000000001), (22, 2, '2022-03-21', 1930.39), (38, 2, '2022-05-21', 2724.65), (9, 2, '2022-05-29', 4020.2200000000003), (85, 2, '2022-05-29', 4020.2200000000003), (30, 2, '2022-06-06', 4983.93), (65, 2, '2022-09-08', 5283.47), (54, 2, '2023-01-12', 5350.96), (48, 2, '2023-01-29', 5716.71), (35, 2, '2023-02-11', 6662.04), (99, 2, '2023-04-24', 7022.0599999999995), (14, 2, '2023-05-03', 7183.49), (66, 2, '2023-07-01', 8162.0599999999995), (4, 2, '2023-07-15', 8322.25), (1, 3, '2021-07-27', 641.68), (33, 3, '2021-08-03', 1427.03), (87, 3, '2021-09-17', 1588.05), (74, 3, '2021-10-02', 2132.2799999999997), (62, 3, '2022-03-23', 2425.8399999999997), (40, 3, '2022-03-30', 3204.04), (94, 3, '2022-04-28', 3

In [22]:
# 4. Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants

cursor = conn.cursor()
cursor.execute('''
    SELECT Products.name, SUM(OrderItems.quantity) AS total_quantity
    FROM Products
    JOIN OrderItems ON Products.product_id = OrderItems.product_id
    GROUP BY Products.name
    ORDER BY total_quantity DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[('MacBook', 399), ('iPhone', 339), ('AirPods', 273), ('Apple Watch', 221), ('iPad', 209)]


In [24]:
conn.close()